# Agent
This notebook covers how to create a custom agent for a chat model. It will utilize chat specific prompts.

In [1]:
import os
os.environ["LANGCHAIN_HANDLER"] = "langchain"

## Uncomment this if using hosted setup.

os.environ["LANGCHAIN_ENDPOINT"] = "http://127.0.0.1:8000" 

from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.chains import LLMChain
from langchain.utilities import SerpAPIWrapper

In [2]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [3]:
prefix = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:"""
suffix = """Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Args"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=[]
)

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [5]:
messages = [
    SystemMessagePromptTemplate(prompt=prompt),
    HumanMessagePromptTemplate.from_template("{input}\n\nThis was your previous work "
                f"(but I haven't seen any of it! I only see what "
                "you return as final answer):\n{agent_scratchpad}")
]

In [6]:
prompt = ChatPromptTemplate.from_messages(messages)

In [7]:
llm_chain = LLMChain(llm=ChatOpenAI(temperature=0), prompt=prompt)

In [8]:
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

In [9]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [10]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
Arrr, ye be in luck, matey! I'll help ye find the answer ye seek.

Thought: Hmm, I don't have this information off the top of me head.
Action: Search
Action Input: "Canada population 2023"

Observation: The current population of Canada is 38,627,607 as of Thursday, March 9, 2023, based on Worldometer elaboration of the latest United Nations data.
Thought:Ahoy, me hearties! I have found the answer to yer question.

Final Answer: As of March 9, 2023, the population of Canada is 38,627,607. Arrr!

> Finished chain.


'As of March 9, 2023, the population of Canada is 38,627,607. Arrr!'